In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
data_url = 'https://raw.githubusercontent.com/andrewpenland/rewardsresearch/main/OceanDAO/final-data/ocean-votes-round-1-to-10.csv'
df = pd.read_csv(data_url, index_col=0)
df.head()

,address,balance,Vote,Round,Project Name,Proposal State,Proposal Standing,Grant Category,Earmarks,OCEAN Granted
0,0xebDcbE0f258B7F8Ebb7d3F5E57faE8C2204a5E64,8862.463109,Yes,1,APY.vision,Not Granted,Not Granted,Build & Integrate,General,0.0
1,0x5D2B315C465e133a346C960F46f5AA1ED88a3179,13840.708383,Yes,1,Data Whale,Funded,Completed,Build & Integrate,General,13000.0
2,0xB121b3DdaA9af45Df9878C855079F8A78eea9772,25934.100000,Yes,1,Data Whale,Funded,Completed,Build & Integrate,General,13000.0
3,0x1911176664F147DCb30DA4a4AA6ef6c6849e613f,4446.742291,Yes,1,Data Whale,Funded,Completed,Build & Integrate,General,13000.0
4,0x45E48C0a6Fe8b759652624451C83387130C58367,20801.288274,Yes,1,Data Whale,Funded,Completed,Build & Integrate,General,13000.0


In [9]:
dfr = df.groupby('Round').apply(lambda x: pd.Series(dict(
    total_addresses = len(x['address']),
    total_votes = sum(x['balance']),
    total_projects = len(np.unique(x['Project Name'])),
    
))).reset_index()

dfr.head(10)

,Round,total_addresses,total_votes,total_projects
0,1,76.0,3.257733e+06,9.0
1,2,81.0,1.195649e+06,14.0
2,3,96.0,3.712733e+06,18.0
3,4,101.0,3.060886e+06,14.0
4,5,246.0,1.067576e+07,15.0
5,6,259.0,1.117569e+07,14.0
6,7,345.0,2.179240e+07,22.0
7,8,369.0,2.141962e+07,18.0
8,9,234.0,1.730843e+07,17.0
9,10,1180.0,1.684062e+08,31.0


In [43]:
df['Proposal State'].value_counts()

Funded         1795
Down Voted      939
Not Granted     159
Granted          94
Name: Proposal State, dtype: int64

In [13]:
df.columns

Index(['address', 'balance', 'Vote', 'Round', 'Project Name', 'Proposal State',
       'Proposal Standing', 'Grant Category', 'Earmarks', 'OCEAN Granted'],
      dtype='object')

In [24]:
dfp = df[['Round', 'Project Name', 'Proposal State',
       'Proposal Standing', 'Grant Category', 'Earmarks', 'OCEAN Granted']].drop_duplicates()

In [25]:
dfp.shape

(173, 7)

In [26]:
dfp.head()

,Round,Project Name,Proposal State,Proposal Standing,Grant Category,Earmarks,OCEAN Granted
0,1,APY.vision,Not Granted,Not Granted,Build & Integrate,General,0.0
1,1,Data Whale,Funded,Completed,Build & Integrate,General,13000.0
15,1,Decentralized File Rating,Funded,Unreported,Build & Integrate,General,13000.0
20,1,Ocean Academy,Funded,Unreported,Outreach,General,13000.0
48,1,Ocean Pool Alerts,Funded,Unreported,Build & Integrate,General,13000.0


In [27]:
dfp['Proposal State'].value_counts()

Funded         109
Down Voted      32
Not Granted     28
Granted          4
Name: Proposal State, dtype: int64

In [31]:
dfp[dfp['Proposal State']=='Granted']

,Round,Project Name,Proposal State,Proposal Standing,Grant Category,Earmarks,OCEAN Granted
1204,8,Ocean Missions,Granted,Completed,Outreach,General,28302.0
1599,9,Resilient ML,Granted,Completed,Unleash Data,General,22764.0
1744,9,Ocean Pearl,Granted,Completed,DAO,NaN,22764.0
2438,10,Governauts Rewards Systems Research,Granted,Completed,DAO,NaN,26527.0


In [32]:
dfp['is_funded'] = dfp['Proposal State'].apply(lambda x: 1 if x in ['Granted', 'Funded'] else 0)

In [33]:
dfp['Proposal Standing'].value_counts()

Completed         94
Unreported        37
Not Granted       25
Down Voted        11
In Progress        3
Funds Returned     2
No Ocean           1
Name: Proposal Standing, dtype: int64

In [39]:
dfp['is_completed'] = dfp.apply(lambda x: 1 if (x['Proposal Standing'] in ['Completed']) and (x['Proposal State'] in ['Granted', 'Funded']) else 0, axis=1)

In [48]:
is_returning = []
exhisting_projects = []
for project in dfp['Project Name'].values:
    if project in exhisting_projects:
        is_returning.append(1)
    else:
        is_returning.append(0)
        exhisting_projects.append(project)
dfp['is_returning'] = is_returning

In [51]:
dfpr = dfp.groupby('Round').apply(lambda x: pd.Series(dict(
    amount_funded = sum(x['OCEAN Granted']),
    projects_submitted = len(x['Project Name']),
    projects_funded = sum(x['is_funded']),
    projects_completed = sum(x['is_completed']),
    projects_returning = sum(x['is_returning'])
))).reset_index()

dfpr.head(10)

,Round,amount_funded,projects_submitted,projects_funded,projects_completed,projects_returning
0,1,65000.0,9.0,5.0,1.0,0.0
1,2,90000.0,14.0,9.0,6.0,3.0
2,3,120000.0,18.0,12.0,5.0,6.0
3,4,91000.0,14.0,12.0,7.0,5.0
4,5,120000.0,16.0,10.0,10.0,8.0
5,6,272000.0,14.0,13.0,9.0,8.0
6,7,400000.0,22.0,14.0,10.0,13.0
7,8,365244.0,18.0,12.0,8.0,12.0
8,9,349001.0,17.0,17.0,15.0,9.0
9,10,158546.0,31.0,9.0,3.0,18.0


In [52]:
dfpr['funded_rate'] = dfpr['projects_funded'] / dfpr['projects_submitted']
dfpr['completed_rate'] = dfpr['projects_completed'] / dfpr['projects_funded']
dfpr['returning_rate'] = dfpr['projects_returning'] / dfpr['projects_submitted']
dfpr

,Round,amount_funded,projects_submitted,projects_funded,projects_completed,projects_returning,funded_rate,completed_rate,returning_rate
0,1,65000.0,9.0,5.0,1.0,0.0,0.555556,0.200000,0.000000
1,2,90000.0,14.0,9.0,6.0,3.0,0.642857,0.666667,0.214286
2,3,120000.0,18.0,12.0,5.0,6.0,0.666667,0.416667,0.333333
3,4,91000.0,14.0,12.0,7.0,5.0,0.857143,0.583333,0.357143
4,5,120000.0,16.0,10.0,10.0,8.0,0.625000,1.000000,0.500000
5,6,272000.0,14.0,13.0,9.0,8.0,0.928571,0.692308,0.571429
6,7,400000.0,22.0,14.0,10.0,13.0,0.636364,0.714286,0.590909
7,8,365244.0,18.0,12.0,8.0,12.0,0.666667,0.666667,0.666667
8,9,349001.0,17.0,17.0,15.0,9.0,1.000000,0.882353,0.529412
9,10,158546.0,31.0,9.0,3.0,18.0,0.290323,0.333333,0.580645
